In [3]:
# My imports

import numpy as np
import pandas as pd
import warnings
import tensorflow as tf

warnings.filterwarnings('ignore')

In [4]:
#tf.test.gpu_device_name()

### Data Preparation

In [5]:
# Load 94 firm characteristics dataset

data_ch = pd.read_csv('/home/dcuiab/ML/GKX_20201231.csv')
data_ch['DATE'] = pd.to_datetime(data_ch['DATE'], format='%Y%m%d') + pd.offsets.MonthEnd(0)
data_ch = data_ch[(data_ch['DATE'] >= '1957-01-31') & (data_ch['DATE'] <= '2016-12-31')]
cols = data_ch.columns.tolist()
cols_new = [x for x in cols if x not in ['permno', 'prc', 'SHROUT', 'mve0']]
data_ch = data_ch[cols_new]
data_ch

,DATE,mvel1,RET,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
306536,1957-01-31,8.224900e+04,0.064378,1.122846,1.260784,0.047180,9.569953,0.025742,0.046197,0.044843,...,0.013234,9.411565e-08,0.015453,0.008058,0.355638,0.460420,1.120996e-07,37.0,NaN,NaN
306537,1957-01-31,3.903375e+03,0.095238,0.426734,0.182102,-0.275641,6.237836,0.069611,0.046197,-0.086957,...,0.033305,6.610609e-06,0.047619,0.033495,1.152126,1.169610,9.229146e-08,NaN,NaN,NaN
306538,1957-01-31,9.273250e+03,0.102041,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046197,-0.060377,...,0.016023,2.286832e-06,0.020833,0.015589,0.815777,0.679803,1.181757e-07,NaN,NaN,NaN
306539,1957-01-31,5.446588e+04,-0.047091,0.926038,0.857547,0.018171,9.825337,0.021700,0.046197,0.044633,...,0.015295,1.464273e-07,0.039326,0.015849,0.739302,1.333656,6.126699e-08,NaN,NaN,NaN
306540,1957-01-31,4.025000e+04,-0.090062,1.247748,1.556875,0.025785,7.901007,0.025506,0.046197,0.086667,...,0.005954,1.380375e-06,0.056856,0.019945,0.755510,0.410391,3.315790e+00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4068670,2016-12-31,1.250976e+06,-0.006024,1.599425,2.558159,-0.137348,14.916210,0.043206,0.017461,-0.001203,...,0.029302,1.010214e-09,0.027604,0.017847,0.490189,5.347476,8.664791e-09,73.0,0.252853,-0.192059
4068671,2016-12-31,5.600537e+06,0.072424,0.298958,0.089376,0.206434,16.319190,0.026199,-0.087838,0.093973,...,0.016001,1.093471e-10,0.040887,0.010437,0.539155,6.252920,8.275139e-09,62.0,0.049733,-0.512336
4068672,2016-12-31,2.473625e+04,-0.127273,2.330889,5.433042,0.328248,9.975208,0.194202,-0.005618,0.279070,...,0.149511,3.889972e-07,0.405473,0.137955,1.285707,29.078756,5.014198e-09,65.0,-1.324437,-1.879364
4068673,2016-12-31,8.573280e+04,-0.041667,0.630433,0.397446,0.020854,10.862196,0.059795,0.015187,-0.049505,...,0.042877,8.576942e-08,0.031579,0.019838,0.629682,2.431419,3.518593e-08,1.0,0.837790,0.244711


In [6]:
# Construct dummy variables

print(data_ch['sic2'].isnull().sum())
data_ch  = data_ch.dropna(subset=['sic2']).reset_index(drop=True)
#print(data_ch['sic2'].isnull().sum())

dummies = pd.get_dummies(data_ch['sic2'], prefix='dum_')
data_ch = data_ch.drop('sic2', axis=1)
data_ch = pd.concat([data_ch, dummies], axis=1)
print(data_ch.shape)

260072
(3502067, 170)


In [7]:
# Fill in missing characteristics

chas = [x for x in cols_new if x not in ['DATE', 'RET', 'sic2']]
#print(chas)
print('Total number of missing characteristics: %d' % (data_ch[chas].isnull().sum().sum()))

for cha in chas:
    data_ch[cha] = data_ch.groupby('DATE')[cha].transform(lambda x: x.fillna(x.median()))
print('Total number of missing characteristics: %d' % (data_ch[chas].isnull().sum().sum()))

for cha in chas:
    data_ch[cha] = data_ch[cha].transform(lambda x: x.fillna(x.median()))
print('Total number of missing characteristics: %d' % (data_ch[chas].isnull().sum().sum()))

Total number of missing characteristics: 86471506
Total number of missing characteristics: 6114874
Total number of missing characteristics: 0


In [8]:
# Load 8 macroeconomic predictors

data_ma = pd.read_csv('/home/dcuiab/ML/PredictorData2023.csv')
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'], format='%Y%m') + pd.offsets.MonthEnd(0)
data_ma = data_ma[(data_ma['yyyymm'] >= '1957-01-31') & (data_ma['yyyymm'] <= '2016-12-31')].reset_index(drop=True)
data_ma

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,1957-01-31,44.72,1.7367,3.4067,0.567243,0.0311,0.0377,0.0449,0.0328,0.027992,0.0027,0.000000,0.0346,0.0197,0.000902,0.000739,-0.040238,-0.041569
1,1957-02-28,43.26,1.7333,3.4033,0.584994,0.0310,0.0367,0.0447,0.0328,0.030173,0.0024,0.003623,0.0025,0.0093,0.001056,0.000827,-0.024919,-0.032823
2,1957-03-31,44.11,1.7300,3.4000,0.599819,0.0308,0.0366,0.0443,0.0331,0.026600,0.0023,0.003610,-0.0024,0.0050,0.000330,0.001054,0.023827,0.020752
3,1957-04-30,45.74,1.7300,3.4067,0.576098,0.0307,0.0367,0.0444,0.0345,0.027421,0.0025,0.003597,-0.0222,-0.0066,0.000302,0.002142,0.046538,0.045215
4,1957-05-31,47.43,1.7300,3.4133,0.564039,0.0306,0.0374,0.0452,0.0348,0.028849,0.0026,0.003584,-0.0023,-0.0075,0.000482,0.002422,0.038734,0.033208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2016-08-31,"2,170.95",44.8371,88.3667,0.315197,0.0030,0.0332,0.0424,0.0186,-0.030786,0.0002,0.000918,-0.0140,0.0016,0.000279,NaN,0.001247,-0.001396
716,2016-09-30,"2,168.27",45.0257,89.0900,0.316794,0.0029,0.0341,0.0431,0.0196,-0.032607,0.0002,0.002404,-0.0124,-0.0119,0.001673,NaN,0.000446,-0.001033
717,2016-10-31,"2,126.15",45.2507,90.9100,0.319688,0.0033,0.0351,0.0438,0.0220,-0.029038,0.0002,0.001247,-0.0314,-0.0263,0.000364,NaN,-0.017944,-0.019193
718,2016-11-30,"2,198.81",45.4756,92.7300,0.303286,0.0045,0.0386,0.0471,0.0267,-0.027456,0.0001,-0.001555,-0.0599,-0.0510,0.000946,NaN,0.035812,0.033029


In [9]:
# Construct 8 macroeconomic predictors

ma_predictors = ['dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar']
data_ma['Index'] = data_ma['Index'].str.replace(',', '').astype('float64')
data_ma['dp'] = np.log(data_ma['D12'] / data_ma['Index'])
data_ma['ep'] = np.log(data_ma['E12'] / data_ma['Index'])
data_ma.rename(columns={'b/m': 'bm'}, inplace=True)
data_ma['tms'] = data_ma['lty'] - data_ma['tbl']
data_ma['dfy'] = data_ma['BAA'] - data_ma['AAA']
data_ma = data_ma[['yyyymm'] + ma_predictors]
data_ma

,yyyymm,dp,ep,bm,ntis,tbl,tms,dfy,svar
0,1957-01-31,-3.248434,-2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902
1,1957-02-28,-3.217201,-2.542483,0.584994,0.030173,0.0310,0.0018,0.0080,0.001056
2,1957-03-31,-3.238565,-2.562911,0.599819,0.026600,0.0308,0.0023,0.0077,0.000330
3,1957-04-30,-3.274852,-2.597229,0.576098,0.027421,0.0307,0.0038,0.0077,0.000302
4,1957-05-31,-3.311134,-2.631575,0.564039,0.028849,0.0306,0.0042,0.0078,0.000482
...,...,...,...,...,...,...,...,...,...
715,2016-08-31,-3.879884,-3.201425,0.315197,-0.030786,0.0030,0.0156,0.0092,0.000279
716,2016-09-30,-3.874451,-3.192038,0.316794,-0.032607,0.0029,0.0167,0.0090,0.001673
717,2016-10-31,-3.849850,-3.152198,0.319688,-0.029038,0.0033,0.0187,0.0087,0.000364
718,2016-11-30,-3.878496,-3.165980,0.303286,-0.027456,0.0045,0.0222,0.0085,0.000946


In [10]:
# Construct the dataset including all covariates

data_ma_long = pd.merge(data_ch['DATE'], data_ma, left_on='DATE', right_on='yyyymm', how='left').drop('yyyymm', axis=1)
for cha in chas:
    for predictor in ma_predictors:
        name = cha + '_' + predictor
        data_ch[name] = data_ch[cha] * data_ma_long[predictor]
data = data_ch
data

,DATE,mvel1,RET,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,bm_dfy,bm_svar,bm_ia_dp,bm_ia_ep,bm_ia_bm,bm_ia_ntis,bm_ia_tbl,bm_ia_tms,bm_ia_dfy,bm_ia_svar
0,1957-01-31,8.224900e+04,0.064378,1.122846,1.260784,0.047180,9.569953,0.025742,0.046197,0.044843,...,0.004545,0.000569,0.320360,0.253914,-0.055941,-0.002761,-0.003067,-0.000168,-0.000710,-0.000089
1,1957-01-31,1.861935e+05,-0.014851,1.445027,2.088104,-0.058910,10.375318,0.025526,0.046197,0.041237,...,0.004545,0.000569,0.320360,0.253914,-0.055941,-0.002761,-0.003067,-0.000168,-0.000710,-0.000089
2,1957-01-31,2.259840e+05,0.009346,0.585546,0.342864,-0.090805,8.887653,0.018691,0.046197,0.018779,...,0.004545,0.000569,0.320360,0.253914,-0.055941,-0.002761,-0.003067,-0.000168,-0.000710,-0.000089
3,1957-01-31,2.798469e+05,-0.032727,1.173314,1.376666,-0.008426,10.547950,0.021665,0.046197,0.117886,...,0.004545,0.000569,0.320360,0.253914,-0.055941,-0.002761,-0.003067,-0.000168,-0.000710,-0.000089
4,1957-01-31,1.940066e+05,-0.031963,1.845969,3.407603,0.051096,9.631990,0.036087,0.046197,0.037915,...,0.004545,0.000569,0.320360,0.253914,-0.055941,-0.002761,-0.003067,-0.000168,-0.000710,-0.000089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3502062,2016-12-31,1.250976e+06,-0.006024,1.599425,2.558159,-0.137348,14.916210,0.043206,0.017461,-0.001203,...,0.001947,0.000133,0.747417,0.607787,-0.056365,0.004822,-0.000980,-0.004245,-0.001479,-0.000101
3502063,2016-12-31,5.600537e+06,0.072424,0.298958,0.089376,0.206434,16.319190,0.026199,-0.087838,0.093973,...,0.000383,0.000026,1.993806,1.621329,-0.150360,0.012864,-0.002613,-0.011323,-0.003945,-0.000268
3502064,2016-12-31,2.473625e+04,-0.127273,2.330889,5.433042,0.328248,9.975208,0.194202,-0.005618,0.279070,...,-0.010198,-0.000694,7.313728,5.947397,-0.551554,0.047187,-0.009585,-0.041534,-0.014471,-0.000985
3502065,2016-12-31,8.573280e+04,-0.041667,0.630433,0.397446,0.020854,10.862196,0.059795,0.015187,-0.049505,...,0.006451,0.000439,-0.952317,-0.774408,0.071818,-0.006144,0.001248,0.005408,0.001884,0.000128


In [11]:
# Split the dataset
# Training set

covariates = [x for x in data.columns if x != 'RET']

train_str = '1957-01-31'; train_end = '1974-12-31'
data_train = data[(train_str <= data['DATE']) & (data['DATE'] <= train_end)]
X_train, y_train = data_train[covariates], data_train[['DATE', 'RET']]
print(X_train.shape); print(y_train.shape)

(345984, 921)
(345984, 2)


In [12]:
# Validation set

val_str = '1975-01-31'; val_end = '1986-12-31'
data_val = data[(val_str <= data['DATE']) & (data['DATE'] <= val_end)]
X_val, y_val = data_val[covariates], data_val[['DATE', 'RET']]
print(X_val.shape); print(y_val.shape)

(680206, 921)
(680206, 2)


In [13]:
# Test set

test_str = '1987-01-31'; test_end = '2016-12-31'
data_test = data[(test_str <= data['DATE']) & (data['DATE'] <= test_end)]
X_test, y_test = data_test[covariates], data_test[['DATE', 'RET']]
print(X_test.shape); print(y_test.shape)

(2475877, 921)
(2475877, 2)
